Assume that we have the following influence diagram and we want to solve it using the jointree algorithm:

![](https://rawgit.com/bhrzslm/notebooks/gh-pages/images/model1.svg)

A quick review of the symbols in the model above:
- stochastic variables:
  - $r_i$: reward associated with item $i$
  - $w_i$: weight of item $i$
  - $h_i$: hidden variable affecting properties of item $i$
- decision variables:
  - $X_i$: binary decision variable indicating whether or not to take item $i$
- utility variables:
  - $U_i$: contribution of item $i$ to the objective function. These variables depend on their parents via this deterministic relation: $U_i = r_i * X_i$
  - $U_p$: a utility of $- \infty$ for assignments that violate the constraint, zero otherwise
- a deterministic variable representing the constraint $\sum_i w_i X_i < C$

Conversion of influence diagrams into jointrees is a bit different from that of Bayesian networks. However I only have the code for converting Bayesian networks to jointrees. So instead of the diagram above, I will compile the following network:

![](https://cdn.rawgit.com/bhrzslm/notebooks/gh-pages/images/bn1.svg)

In [1]:
push!(LOAD_PATH, pwd())
using JoinTreeInference.Node
using JoinTreeInference.Potential
using JoinTreeInference.parse_net
using JoinTreeInference.create_moral_graph
using JoinTreeInference.triangulate_graph
using JoinTreeInference.create_junction_tree

The first step is to create the moral graph of the Bayesian network

In [2]:
nodes, potentials = parse_net("./bn.net")
g_moral = create_moral_graph(nodes, potentials)

Dict{String,Set{String}} with 12 entries:
  "h1" => Set(String["r1","h2","w1"])
  "c"  => Set(String["x1","x2","w1","w2","up"])
  "w1" => Set(String["h1","x1","c","x2","w2"])
  "u1" => Set(String["x1","r1"])
  "up" => Set(String["c"])
  "h2" => Set(String["h1","r2","w2"])
  "x2" => Set(String["x1","c","r2","w1","w2","u2"])
  "w2" => Set(String["x1","c","h2","x2","w1"])
  "u2" => Set(String["r2","x2"])
  "x1" => Set(String["c","r1","x2","w1","w2","u1"])
  "r1" => Set(String["h1","x1","u1"])
  "r2" => Set(String["h2","x2","u2"])

This is what the moral graph looks like:

![](https://rawgit.com/bhrzslm/notebooks/gh-pages/images/bn_moral.svg)

Next, we triangulate the moral graph:

In [7]:
g_triangulated, clusters = triangulate_graph(g_moral, nodes)
g_triangulated

Dict{String,Set{String}} with 12 entries:
  "h1" => Set(String["x1","r1","h2","w1"])
  "c"  => Set(String["x1","x2","w1","w2","up"])
  "w1" => Set(String["h1","x1","c","h2","x2","w2"])
  "u1" => Set(String["x1","r1"])
  "up" => Set(String["c"])
  "h2" => Set(String["h1","x1","r2","x2","w2","w1"])
  "x2" => Set(String["x1","c","r2","h2","w1","w2","u2"])
  "w2" => Set(String["x1","c","h2","x2","w1"])
  "u2" => Set(String["r2","x2"])
  "x1" => Set(String["h1","c","r1","h2","x2","w1","w2","u1"])
  "r1" => Set(String["h1","x1","u1"])
  "r2" => Set(String["h2","x2","u2"])

Ok. So triangulation adds some new edges:

![](https://rawgit.com/bhrzslm/notebooks/gh-pages/images/bn_triangulated.svg)

We also collected the clusters (that is, the maximal cliques of the triangulated graph) during the triangulation step:

In [5]:
clusters

8-element Array{Set{String},1}:
 Set(String["c","up"])                
 Set(String["x1","r1","u1"])          
 Set(String["r2","x2","u2"])          
 Set(String["x1","c","x2","w1","w2"]) 
 Set(String["h1","x1","r1"])          
 Set(String["r2","h2","x2"])          
 Set(String["h1","x1","h2","w1"])     
 Set(String["x1","h2","x2","w2","w1"])

These are the 8 clusters:

![](https://rawgit.com/bhrzslm/notebooks/gh-pages/images/clusters.svg)

It is difficult to see the clusters in the image above. So I removed the edges:

![](https://rawgit.com/bhrzslm/notebooks/gh-pages/images/clusters2.svg)

Now we can create the jointree (= junction tree = cluster tree)

In [6]:
create_junction_tree(clusters, nodes)

Dict{Int64,Set{Int64}} with 8 entries:
  7 => Set([8,5])
  4 => Set([8,1])
  2 => Set([5])
  3 => Set([6])
  5 => Set([7,2])
  8 => Set([7,4,6])
  6 => Set([3,8])
  1 => Set([4])

Each number in the list above is the index of a cluster in the `clusters` arrray. This is what the tree looks like:

![](https://rawgit.com/bhrzslm/notebooks/gh-pages/images/jointree1.svg)

First observation: the nodes $X_1, X_2, w_1, w_2, C$ have appeared in the same cluster. For my knapsack problem I expect this to be always the case, that is to have all $X_i, w_i, C$ in one cluster. The reason is that they are part of a factor, and the variables in each factor in the Bayesian network should be subset of variables of some cluster in the jointree. 

Looking at the steps in creation of jointree, this is reasonable I guess. The variables in a factor form a clique even before triangulation. So they will all be together in some maximal clique (which is some node in the jointree).